# 卷积神经网络

## 二维互相关运算

In [1]:
import torch
from  torch import nn

def corr2D(X,kernel):
    h,w = kernel.shape
    Y = torch.zeros(size=(X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*kernel).sum()
    return Y

## 卷积层

In [2]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight = torch.rand(kernel_size)
        self.bias = torch.zeros(1)

    def forward(self,X):
        return corr2D(X,self.weight)+self.bias

## 训练过程
卷积神经网络训练的权值就是卷积核，比如本例中通过类似Sobel算子的卷积核进行滤波后得出原图像上的边界，而训练就是尝试训练出一个卷积核类似Sobel算子，同样能识别边界

In [3]:
X = torch.ones((6, 8))
X[:,2:6] = 0
kernel = torch.tensor([[1,-1]])
Y = corr2D(X,kernel)

conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X = X.reshape(1,1,6,8) # 批量大小、通道、高度、宽度
Y = Y.reshape(1,1,6,7)
lr = 0.03

for i in range(10):
    y_hat = conv2d(X)
    loss = (y_hat-Y)**2
    conv2d.zero_grad()
    loss.sum().backward()
    conv2d.weight.data[:] -= lr*conv2d.weight.grad
    print(f'epoch{i} loss:{loss.sum()}')

print(conv2d.weight.data)

epoch0 loss:13.110658645629883
epoch1 loss:6.3352556228637695
epoch2 loss:3.212603807449341
epoch3 loss:1.7111999988555908
epoch4 loss:0.9539106488227844
epoch5 loss:0.5526436567306519
epoch6 loss:0.3299928307533264
epoch7 loss:0.20148806273937225
epoch8 loss:0.12497634440660477
epoch9 loss:0.07835618406534195
tensor([[[[ 0.9617, -1.0184]]]])
